In [1]:
# Imports
import ffmpeg
import pandas as pd
import geopandas
import datetime
import shapely
import warnings
import numpy as np
import os
from pathlib import Path

import video_transect, misc

In [2]:
# Paths
csv_file = '/media/mha114/MassimalDataProcessing/20210312_Saltstraumen/Images-UW/20220329/VideoTransect_Structured/Transect2/2022-03-29_135354_MassimalTrack.csv'
video_dir = '/media/mha114/MassimalDataProcessing/20210312_Saltstraumen/Images-UW/20220329/VideoTransect_Structured/Transect2/Video'
image_dir = '/media/mha114/MassimalDataProcessing/20210312_Saltstraumen/Images-UW/20220329/VideoTransect_Structured/Transect2/Images'
gpk_file = '/media/mha114/MassimalDataProcessing/20210312_Saltstraumen/Images-UW/20220329/VideoTransect_Structured/Transect2/20210312_Saltstraumen_VideoTransect2.gpkg'

In [3]:
# Read file, get geodataframe
gdf = video_transect.track_csv_to_geodataframe(csv_file)

In [5]:
# Get video info
video_data = video_transect.get_video_data(video_dir)

In [6]:
# Prepare gdf for extraction of images
gdf = video_transect.prepare_gdf_with_video_data(gdf,video_data)

In [7]:
# Ony keep every n'th row (too densely sampled)
N = 5  # Every fifth second
gdf_small = gdf.copy()
gdf_small = gdf_small.iloc[::N]

In [8]:
gdf_small

,Lat,Lng,Time,TimeDiffSec,VideoFile,VideoRelTime,geometry
0,67.227312,14.625168,2022-03-29 11:53:55.088000+00:00,0.000,/media/mha114/MassimalDataProcessing/20210312_...,0.000,POINT (14.62517 67.22731)
5,67.227311,14.625168,2022-03-29 11:53:59.068000+00:00,3.980,/media/mha114/MassimalDataProcessing/20210312_...,3.980,POINT (14.62517 67.22731)
10,67.227317,14.625164,2022-03-29 11:54:04.105000+00:00,9.017,/media/mha114/MassimalDataProcessing/20210312_...,9.017,POINT (14.62516 67.22732)
15,67.227312,14.625166,2022-03-29 11:54:09.250000+00:00,14.162,/media/mha114/MassimalDataProcessing/20210312_...,14.162,POINT (14.62517 67.22731)
20,67.227310,14.625162,2022-03-29 11:54:14.109000+00:00,19.021,/media/mha114/MassimalDataProcessing/20210312_...,19.021,POINT (14.62516 67.22731)
...,...,...,...,...,...,...,...
940,67.227981,14.623419,2022-03-29 12:09:34.129000+00:00,939.041,/media/mha114/MassimalDataProcessing/20210312_...,406.509,POINT (14.62342 67.22798)
945,67.227942,14.623411,2022-03-29 12:09:39.126000+00:00,944.038,/media/mha114/MassimalDataProcessing/20210312_...,411.506,POINT (14.62341 67.22794)
950,67.227901,14.623411,2022-03-29 12:09:44.144000+00:00,949.056,/media/mha114/MassimalDataProcessing/20210312_...,416.524,POINT (14.62341 67.22790)
955,67.227860,14.623417,2022-03-29 12:09:49.127000+00:00,954.039,/media/mha114/MassimalDataProcessing/20210312_...,421.507,POINT (14.62342 67.22786)


In [9]:
# Extract images
gdf_small = video_transect.extract_images_from_video(gdf_small,image_dir)

100%|█████████████████████████████████████████| 193/193 [02:14<00:00,  1.44it/s]


In [10]:
gdf_small.tail()

,Lat,Lng,Time,TimeDiffSec,VideoFile,VideoRelTime,ImageFile,geometry
940,67.227981,14.623419,2022-03-29 12:09:34.129000+00:00,939.041,/media/mha114/MassimalDataProcessing/20210312_...,406.509,GX020371_06m46s509ms.jpg,POINT (14.62342 67.22798)
945,67.227942,14.623411,2022-03-29 12:09:39.126000+00:00,944.038,/media/mha114/MassimalDataProcessing/20210312_...,411.506,GX020371_06m51s506ms.jpg,POINT (14.62341 67.22794)
950,67.227901,14.623411,2022-03-29 12:09:44.144000+00:00,949.056,/media/mha114/MassimalDataProcessing/20210312_...,416.524,GX020371_06m56s524ms.jpg,POINT (14.62341 67.22790)
955,67.227860,14.623417,2022-03-29 12:09:49.127000+00:00,954.039,/media/mha114/MassimalDataProcessing/20210312_...,421.507,GX020371_07m01s507ms.jpg,POINT (14.62342 67.22786)
960,67.227824,14.623451,2022-03-29 12:09:54.103000+00:00,959.015,/media/mha114/MassimalDataProcessing/20210312_...,426.483,GX020371_07m06s483ms.jpg,POINT (14.62345 67.22782)


In [11]:
# Save gdf as geopackage
gdf_small.to_file(gpk_file, driver="GPKG")